In [1]:
## Stacking Grasps!
import pandas as pd 
import numpy as np 
import pickle

In [2]:
from sklearn.svm import SVC, OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier as RFC, IsolationForest as IF

In [3]:
from lightgbm import LGBMClassifier as LGBC
from xgboost import XGBClassifier as XGBC

In [4]:
## Loading Model Results
data_dir = "/media/gvk/GVK_Elements/github_repo/Stacking_Grasp/Merged_Results_AllTypes.csv"
results_df= pd.read_csv(data_dir)

In [5]:
results_df

,Unnamed: 0,Grasp_ID,Color_IoU,Color_Positive,Depth_IoU,Depth_Positive,GGCNN2_IoU,GGCNN2_Positive,GGCNN_IoU,GGCNN_Positive,Color_Negative,Depth_Negative,GGCNN2_Negative,GGCNN_Negative,Grasp_Type
0,0,Positive_pcd0100_ModelGrasp_0_GT_Grasp_0,0.283525,True,0.000000,False,0.000000,False,0.000000,False,False,False,False,False,1
1,1,Positive_pcd0100_ModelGrasp_0_GT_Grasp_1,0.448869,True,0.111162,False,0.056329,False,0.111162,False,False,False,False,False,1
2,2,Positive_pcd0100_ModelGrasp_0_GT_Grasp_2,0.048695,False,0.343457,True,0.249125,False,0.343457,True,False,False,False,False,1
3,3,Positive_pcd0100_ModelGrasp_0_GT_Grasp_3,0.000000,False,0.355614,True,0.436653,True,0.355614,True,False,False,False,False,1
4,4,Positive_pcd0101_ModelGrasp_0_GT_Grasp_0,0.000000,False,0.019095,False,0.000000,False,0.019095,False,False,False,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7225,2674,Negative_pcd1033_ModelGrasp_0_GT_Grasp_4,0.000000,False,0.000000,False,0.000000,False,0.000000,False,False,False,False,False,-1
7226,2675,Negative_pcd1034_ModelGrasp_0_GT_Grasp_0,0.503114,False,0.319620,False,0.226308,False,0.319620,False,True,True,False,True,-1
7227,2676,Negative_pcd1034_ModelGrasp_0_GT_Grasp_1,0.000000,False,0.000000,False,0.000000,False,0.000000,False,False,False,False,False,-1
7228,2677,Negative_pcd1034_ModelGrasp_0_GT_Grasp_2,0.000000,False,0.000000,False,0.000000,False,0.000000,False,False,False,False,False,-1


In [6]:
X = results_df.drop(columns = ['Unnamed: 0', 'Grasp_ID', 'Grasp_Type'])
y = results_df['Grasp_Type']

In [7]:
X

,Color_IoU,Color_Positive,Depth_IoU,Depth_Positive,GGCNN2_IoU,GGCNN2_Positive,GGCNN_IoU,GGCNN_Positive,Color_Negative,Depth_Negative,GGCNN2_Negative,GGCNN_Negative
0,0.283525,True,0.000000,False,0.000000,False,0.000000,False,False,False,False,False
1,0.448869,True,0.111162,False,0.056329,False,0.111162,False,False,False,False,False
2,0.048695,False,0.343457,True,0.249125,False,0.343457,True,False,False,False,False
3,0.000000,False,0.355614,True,0.436653,True,0.355614,True,False,False,False,False
4,0.000000,False,0.019095,False,0.000000,False,0.019095,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
7225,0.000000,False,0.000000,False,0.000000,False,0.000000,False,False,False,False,False
7226,0.503114,False,0.319620,False,0.226308,False,0.319620,False,True,True,False,True
7227,0.000000,False,0.000000,False,0.000000,False,0.000000,False,False,False,False,False
7228,0.000000,False,0.000000,False,0.000000,False,0.000000,False,False,False,False,False


In [98]:
## Data Preprocessing
# lenX = len(X)
# ratio = int(0.9 * lenX)
# inds = np.random.permutation(np.arange(lenX))
# trainX, testX = X[:ratio], X[ratio:]
# trainY, testY = y[:ratio], y[ratio:]
random_state = 42
test_size = 0.01
trainX, testX, trainY, testY =  train_test_split(X, y, test_size = test_size, random_state = random_state)

In [99]:
print("Test_size: ", test_size)
results_SVC = SVC(random_state = random_state,  probability = True)
results_SVC.fit(X = trainX, y = trainY)
predictY = results_SVC.predict(testX)
print(classification_report(testY, predictY))

Test_size:  0.01
              precision    recall  f1-score   support

          -1       0.44      0.84      0.58        19
           1       0.92      0.63      0.75        54

    accuracy                           0.68        73
   macro avg       0.68      0.74      0.66        73
weighted avg       0.80      0.68      0.70        73



In [100]:
## LGBM
print("Test_size: ", test_size)
results_LGBC = LGBC(random_state = random_state)
results_LGBC.fit(X = trainX, y = trainY)
predictY_LGBC = results_LGBC.predict(X = testX)
print(classification_report(testY, predictY_LGBC))

Test_size:  0.01
              precision    recall  f1-score   support

          -1       0.50      0.79      0.61        19
           1       0.91      0.72      0.80        54

    accuracy                           0.74        73
   macro avg       0.70      0.76      0.71        73
weighted avg       0.80      0.74      0.75        73



In [101]:
## XGBoost
print("Test_size: ", test_size)
results_XGBC = XGBC(random_state = random_state)
results_XGBC.fit(X = trainX, y = trainY)
predictY_XGBC = results_XGBC.predict(testX)
print(classification_report(testY, predictY_XGBC))

Test_size:  0.01
[02:41:44] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
              precision    recall  f1-score   support

          -1       0.50      0.84      0.63        19
           1       0.93      0.70      0.80        54

    accuracy                           0.74        73
   macro avg       0.71 

In [102]:
## Random Forest
print("Test_size: ", test_size)
results_RFC = RFC(random_state = random_state)
results_RFC.fit(X = trainX, y = trainY)
predictY_RFC = results_RFC.predict(X = testX)
print(classification_report(testY, predictY_RFC))

Test_size:  0.01
              precision    recall  f1-score   support

          -1       0.48      0.84      0.62        19
           1       0.93      0.69      0.79        54

    accuracy                           0.73        73
   macro avg       0.70      0.76      0.70        73
weighted avg       0.81      0.73      0.74        73

